<a href="https://colab.research.google.com/github/VICKYCREATIONS/AI-CHATBOT-WITH-NLP/blob/main/IMDb_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow-datasets


In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Correct loading
data, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split dataset
train_data = data['train']
test_data = data['test']

print("✅ IMDb dataset loaded successfully!")

✅ IMDb dataset loaded successfully!


In [8]:
from tensorflow.keras.layers import TextVectorization

# Define parameters
VOCAB_SIZE = 10000         # Top 10,000 words to remember
SEQUENCE_LENGTH = 200      # Max length of sentence

# Create vectorizer layer
vectorizer = TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=SEQUENCE_LENGTH)

# Apply vectorizer to training text
train_text = train_data.map(lambda text, label: text)
vectorizer.adapt(train_text)

print("✅ Text vectorization completed!")

✅ Text vectorization completed!


In [9]:
from tensorflow.keras import layers, models

# Build the model
model = models.Sequential([
    layers.Embedding(input_dim=10000, output_dim=16),               # word vector (16-d)
    layers.GlobalAveragePooling1D(),                                # reduce dimension
    layers.Dense(16, activation='relu'),                            # hidden layer
    layers.Dense(1, activation='sigmoid')                           # output: 0 or 1
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# See summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Convert text reviews to vectorized form using the vectorizer
train_ds = train_data.map(lambda text, label: (vectorizer(text), label))
test_ds = test_data.map(lambda text, label: (vectorizer(text), label))

# Batch the data for training
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [12]:
# Train the model for 5 epochs
history = model.fit(train_ds, validation_data=test_ds, epochs=5)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.6707 - loss: 0.6132 - val_accuracy: 0.8400 - val_loss: 0.3823
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 27ms/step - accuracy: 0.8689 - loss: 0.3253 - val_accuracy: 0.8570 - val_loss: 0.3380
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.8980 - loss: 0.2582 - val_accuracy: 0.8618 - val_loss: 0.3313
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 26ms/step - accuracy: 0.9134 - loss: 0.2217 - val_accuracy: 0.8614 - val_loss: 0.3374
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9246 - loss: 0.1953 - val_accuracy: 0.8581 - val_loss: 0.3522


In [13]:
# Evaluate model on test dataset
loss, accuracy = model.evaluate(test_ds)

print(f"\n✅ Test Accuracy: {accuracy * 100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8587 - loss: 0.3537

✅ Test Accuracy: 85.81%


In [16]:
def predict_sentiment(review):
    # Step 1: Convert review to tensor
    review_tensor = tf.convert_to_tensor([review])

    # Step 2: Vectorize the review
    vectorized_review = vectorizer(review_tensor)

    # Step 3: Predict using the model
    prediction = model.predict(vectorized_review)[0][0]

    # Step 4: Return result
    if prediction > 0.5:
        return f"😊 Positive Review (Confidence: {prediction:.2f})"
    else:
        return f"😞 Negative Review"
print(predict_sentiment("This movie was an absolute masterpiece!"))
print(predict_sentiment("I didn’t like the film at all. It was boring."))
print(predict_sentiment("It was okay, not too bad."))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
😊 Positive Review (Confidence: 0.64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
😞 Negative Review
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
😞 Negative Review
